In [ ]:
# !pip install pyvi
# !pip install dask
# !pip install num2words
# !pip install spacy
# !python -m spacy download en_core_web_sm

# !pip install dask


- References:
  1. Preprocessing: https://www.studocu.com/vn/document/truong-dai-hoc-cong-nghe-thong-tin-va-truyen-thong-viet-han/vi-dieu-khien/danang-nlp-pre-processing-vietnamese/72944823
  2. https://huggingface.co/madha98/Text_Generation_LSTM/blob/main/automatic-text-generation-lstm5.ipynb
  3. https://www.kaggle.com/code/prashant111/a-beginners-guide-to-dealing-with-text-data#4.6-Sentiment-Analysis--
  5. https://www.kaggle.com/code/shivamb/beginners-guide-to-text-generation-using-lstms/notebook#3.-Dataset-preparation
  6. https://nguyenvanhieu.vn/phan-loai-van-ban-tieng-viet/#ftoc-heading-4
  7. https://github.com/trannguyenhan/preprocessing-data/blob/ipynb/preprocessing_data.ipynb
  8. https://www.analyticsvidhya.com/blog/2022/02/explaining-text-generation-with-lstm/
  9. https://www.researchgate.net/publication/321259272_Multi-channel_LSTM-CNN_model_for_Vietnamese_sentiment_analysis (đọc sau)

In [4]:
from pyvi import ViTokenizer
import os
from tqdm import tqdm
import numpy as np
import time
import string
from num2words import num2words as n2w 
import spacy 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

import shutil
from dask import delayed, compute
from dask.bag import Bag

In [ ]:
# TEST 1 file

# Read toàn bộ nội dung trong file .txt (A)
def read_output_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()  
    return content

output_file_path = '/kaggle/working/processed_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'

file_content = read_output_file(output_file_path)

# Dùng pandas để read file, bỏ qua các dòng lỗi ==> kết quả là phần header, title, footer (B)
df_4cogai = pd.read_csv(
    output_file_path,
    encoding='utf-8',
    sep='\t',
    on_bad_lines='skip'
)

# Lấy các dòng từ (A) dưới dạng list
lines_from_df = df_4cogai.astype(str).apply(lambda x: '\t'.join(x), axis=1).tolist()

# Lấy các dòng có trong A nhưng không có trong B. => Phần này em test với tầm 5 files thì thấy kết quả giữ lại phần nội dung, loại bỏ các phần như Chương 1, Chương 2, ... phần source ở dưới mỗi file.
# => Vì không nghĩ ra được rule để clean những phần đó trong mỗi file .txt nên em dùng cách này, chắc là trick thôi
lines_not_in_df = [line.strip() for line in file_content if line.strip() not in lines_from_df]
for line in lines_not_in_df:
    print(line)


In [107]:
# Apply for all files, clean header, chapter & footer content

def read_output_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
    return content

def process_file(file_path, output_folder):
    try:
        file_content = read_output_file(file_path)
        
        df = pd.read_csv(file_path, encoding='utf-8', sep='\t', on_bad_lines='skip')
        
        lines_from_df = df.astype(str).apply(lambda x: '\t'.join(x), axis=1).tolist()
        
        lines_not_in_df = [line.strip() for line in file_content if line.strip() not in lines_from_df]

        output_file_path = os.path.join(output_folder, os.path.basename(file_path))
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            for line in lines_not_in_df:
                output_file.write(line + '\n')

    except Exception as e: # Try except để kiểm tra có file nào không apply được cách này không, báo lỗi để check tiếp
        print(f"Error processing {file_path}: {e}")

input_folder = '/kaggle/input/10000-vietnamese-books/output'
output_folder = '/kaggle/working/cleaned_header_footer_books' # lưu lại 1 file để lưu trữ, e commit file Kaggle bị failed, nên bị mất file ở output session rồi

# nếu chưa có folder sẵn thì tạo folder mới
os.makedirs(output_folder, exist_ok=True)

# Tính thời gian chạy
start_time = time.time()

file_list = [f for f in os.listdir(input_folder) if f.endswith('.txt')]
for file_name in tqdm(file_list, desc="Processing files"):
    file_path = os.path.join(input_folder, file_name)
    process_file(file_path, output_folder)

# Tính thời gian chạy
end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds.") # Thời gian chạy 


Processing files: 100%|██████████| 10415/10415 [02:04<00:00, 83.95it/s] 

Processing completed in 124.09 seconds.


In [110]:
# Check random 1 file sau khi clean header & footer (4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt)
def read_output_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

output_file_path = '/kaggle/working/cleaned_header_footer_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'

file_content = read_output_file(output_file_path)

print(file_content[:100]) # Phần header và tên các chương ko còn

print(file_content[-100:]) # Phần footer chứa source,... đã clean


Lễ Giáng sinh sẽ không phải là lễ Giáng sinh nếu chúng ta không có quà. - Jo vừa lẩm bẩm vừa nằm dà
 hôn nhân hạnh phúc cũng đủ để kết thúc vui vẻ một câu chuyện từng trải qua những giờ phút đen tối.



In [73]:
import spacy
# Tải mô hình SpaCy cho tiếng Anh
nlp = spacy.load('en_core_web_sm') # dùng để clean tên riêng tiếng Anh xuất hiện trong các files

In [76]:
# source: https://github.com/HienBM/sentiment_analysis_with_deep_learning/blob/main/Vietnamese_Proper_Nouns.txt
vietnamese_proper_nouns_path = '/kaggle/input/d/quynhnhitran02/vietnamese-proper-nouns/Vietnamese_Proper_Nouns.txt' # clean tên riêng tiếng Việt xuất hiện trong các files

In [112]:
# Pre processing test với 1 file 4 cô con gái nhà bác sỹ March - Louisa May Alcott.tx

# load_stopwords
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

# load_vietnamese_proper_nouns
def load_vietnamese_proper_nouns(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

# normalize_text
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Thay thế nhiều khoảng trắng thành một khaongr trắng
    text = re.sub(r'[,.]', ' ', text)  # Thay thế dấu . và , bằng khoảng trắng (Để từ cuối cùng của câu và từ bắt đầu của câu không bị dính vào nhau)
    text = re.sub(r'[^\w\s]', '', text)  # Xóa ký tự không phải chữ và không phải số 
    text = re.sub(r'(\d+)', '<number>', text)  # Thay số bằng '<number>'
    return text

# remove_english_proper_nouns
def remove_english_proper_nouns(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not (token.ent_type_ == "PERSON")]
    return ' '.join(filtered_tokens)

# remove_vietnamese_proper_nouns
def remove_vietnamese_proper_nouns(text, proper_nouns):
    for noun in proper_nouns:
        text = text.replace(noun, '')
    return text

# normalize_vietnamese_text
def normalize_vietnamese_text(text):
    text = ViTokenizer.tokenize(text)  # Token hóa văn bản
    text = re.sub(r'\u200B', '', text)  # Xóa ký tự không nhìn thấy 
    return text.replace(' ', '')  # Xóa khoảng trắng

# Hàm chuyển đổi số thành chữ
def lemmatize_numbers(text):
    return re.sub(r'(\d+)', lambda x: n2w(int(x.group())), text)  # Chuyển số thành chữ

# Hàm token hóa văn bản tiếng Việt
def tokenize_vietnamese_text(text):
    return ViTokenizer.tokenize(text)
    
# Hàm xóa từ trùng lặp
def remove_duplicates(tokens):
    seen = set()  # Tạo một set để theo dõi các từ đã gặp
    unique_tokens = []
    for token in tokens.split():
        if token not in seen:
            unique_tokens.append(token)
            seen.add(token)
    return ' '.join(unique_tokens)

# Hàm tiền xử lý văn bản đầy đủ
def preprocess_text(text, stop_words, proper_nouns):
    text = normalize_text(text)
    text = remove_english_proper_nouns(text)  # Xóa tên riêng tiếng Anh
    text = remove_vietnamese_proper_nouns(text, proper_nouns)  # Xóa tên riêng tiếng Việt
    tokens = tokenize_vietnamese_text(text)  # Token hóa văn bản
    tokens = [word for word in tokens.split() if word not in stop_words]  # Loại bỏ stopwords
    tokens = remove_duplicates(' '.join(tokens))  # Loại bỏ từ trùng lặp
    text = lemmatize_numbers(tokens)  # Chuyển đổi số thành chữ
    return text

# Hàm xử lý file văn bản
def process_text_file(input_file, output_file, stop_words, proper_nouns):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        processed_text = preprocess_text(content, stop_words, proper_nouns)

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(processed_text)
            
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Đường dẫn đến file đầu vào và đầu ra
input_file_path = '/kaggle/working/cleaned_header_footer_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'
output_file_path = '/kaggle/working/test_preprocessing_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'
stopwords_path = '/kaggle/input/vietnamesestopwordstxt/vietnamese-stopwords.txt'
vietnamese_proper_nouns_path = '/kaggle/input/d/quynhnhitran02/vietnamese-proper-nouns/Vietnamese_Proper_Nouns.txt'  # Đường dẫn đến file tên riêng tiếng Việt

# Tải stopwords và tên riêng tiếng Việt
stop_words = load_stopwords(stopwords_path)
vietnamese_proper_nouns = load_vietnamese_proper_nouns(vietnamese_proper_nouns_path)

# Thời gian bắt đầu
start_time = time.time()

# Xử lý file
with tqdm(total=1, desc="Processing files") as pbar:  # Khởi tạo tqdm cho toàn bộ quy trình
    process_text_file(input_file_path, output_file_path, stop_words, vietnamese_proper_nouns)
    pbar.update(1)  # Cập nhật tiến trình sau khi xử lý xong file

# Thời gian kết thúc
end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds.")


Processing files: 100%|██████████| 1/1 [00:09<00:00,  9.41s/it]

Processing completed in 9.41 seconds.


In [118]:
# Hàm tải stopwords từ file
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

# Hàm đọc danh sách tên riêng tiếng Việt
def load_vietnamese_proper_nouns(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

# Hàm chuẩn hóa văn bản
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Thay thế nhiều khoảng trắng thành một
    text = re.sub(r'[,.]', ' ', text)  # Thay thế dấu . và , bằng khoảng trắng
    text = re.sub(r'[^\w\s]', '', text)  # Xóa ký tự không phải chữ và không phải số
    text = re.sub(r'(\d+)', '<number>', text)  # Thay số bằng '<number>'
    return text

# Hàm xóa tên riêng tiếng Anh
def remove_english_proper_nouns(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not (token.ent_type_ == "PERSON")]
    return ' '.join(filtered_tokens)

# Hàm xóa tên riêng tiếng Việt
def remove_vietnamese_proper_nouns(text, proper_nouns):
    for noun in proper_nouns:
        text = text.replace(noun, '')
    return text

# Hàm chuẩn hóa văn bản tiếng Việt
def normalize_vietnamese_text(text):
    text = ViTokenizer.tokenize(text)  # Token hóa văn bản
    text = re.sub(r'\u200B', '', text)  # Xóa ký tự không nhìn thấy
    return text.replace(' ', '')  # Xóa khoảng trắng

# xóa các từ tiếng anh xuất hiện trong các văn bản: đại_học copenhagen đan mạch nguyên_bản hội nghiên_cứu đảo đông paris giáo tế california asia

# chuyển số thành chữ
# def lemmatize_numbers(text):
#     return re.sub(r'(\d+)', lambda x: n2w(int(x.group())), text)  # Chuyển số thành chữ

# Dùng thư viện ViTokenizer để tokenize
def tokenize_vietnamese_text(text):
    return ViTokenizer.tokenize(text)
    
# Hàm các xóa từ trùng lặp trong văn bản: ví dụ lễ giáng sinh, chúng ta,...
def remove_duplicates(tokens):
    seen = set()  # tạo set để theo dõi các từ đã gặp
    unique_tokens = []
    for token in tokens.split():
        if token not in seen:
            unique_tokens.append(token)
            seen.add(token)
    return ' '.join(unique_tokens)

# preprocess_text
def preprocess_text(text, stop_words, proper_nouns):
    text = normalize_text(text)
    text = remove_english_proper_nouns(text)  # Xóa tên riêng tiếng Anh
    text = remove_vietnamese_proper_nouns(text, proper_nouns)  # Xóa tên riêng tiếng Việt
    tokens = tokenize_vietnamese_text(text)  # Token hóa văn bản
    tokens = [word for word in tokens.split() if word not in stop_words]  # Loại bỏ stopwords
    tokens = remove_duplicates(' '.join(tokens))  # Loại bỏ từ trùng lặp
    text = lemmatize_numbers(tokens)  # Chuyển đổi số thành chữ
    return text

# process_text_file
def process_text_file(input_file, output_file, stop_words, proper_nouns):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)  # tạo folder để lưu kết quả nếu chưa tồn tại
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        processed_text = preprocess_text(content, stop_words, proper_nouns)

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(processed_text)
            
    except Exception as e:
        print(f"Error processing {input_file}: {e}")

input_file_path = '/kaggle/working/cleaned_header_footer_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'
output_file_path = '/kaggle/working/test_preprocessing_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'
stopwords_path = '/kaggle/input/vietnamesestopwordstxt/vietnamese-stopwords.txt'
vietnamese_proper_nouns_path = '/kaggle/input/d/quynhnhitran02/vietnamese-proper-nouns/Vietnamese_Proper_Nouns.txt'  # Đường dẫn đến file tên riêng tiếng Việt

# Tải stopwords và tên riêng tiếng Việt
stop_words = load_stopwords(stopwords_path)
vietnamese_proper_nouns = load_vietnamese_proper_nouns(vietnamese_proper_nouns_path)

# Thời gian bắt đầu
start_time = time.time()

# Xử lý file
with tqdm(total=1, desc="Processing files") as pbar:  # dùng tqdm check progress 
    process_text_file(input_file_path, output_file_path, stop_words, vietnamese_proper_nouns)
    pbar.update(1)  # cập nhật progress sau khi xử lý xong file

# Thời gian kết thúc
end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds.")

Processing files: 100%|██████████| 1/1 [00:09<00:00,  9.38s/it]

Processing completed in 9.39 seconds.


In [119]:
# kiểm tra kết quả sau khi pre processing

processed_4cogai_path = '/kaggle/working/test_preprocessing_books/4 cô con gái nhà bác sỹ March - Louisa May Alcott.txt'

processed_4cogai = read_output_file(processed_4cogai_path)

print(processed_4cogai[:400])

print(processed_4cogai[-400:])

lễ giáng_sinh chúng_ta quà jo lẩm_bẩm nằm thảm khủng_khiếp ta giàu meg thở_dài áo có_thể công_bằng con_gái xinh_đẹp đứa chẳng bé amy tiếp_lời bấy_giờ vọng góc may_mắn mẹ thân_yêu bốn chị_em hòa_hợp quotcó thể bao_giờ quot trận đánh miền nam quốc đối_lập đồng_hồ treo tường sáu hơ đôi lửa mòn_vẹt bảo mua ý_kiến đáp vỗ thưởng vui lắm giọng vang ngưỡng_cửa sung_sướng đón phụ_nữ dáng_vẻ dễ_thương khuôn
kiện trấn hưng vợ_chồng mơ_ước vờivà ngỏ cầu_hôn ruộng rừng_núi bò bê dê đỏm_dáng cày ghế_dài nhẹn chụp đâu_đây đâytôi chứluôn diệu_kỳ xanh_rì rừng trang_trại vô_số con_con vịt đực heo xu chương_trình lúa đài cỏn_con laurentzvậy nhà_nông chính_cống hớn_hở trang_trọng khẳng_định chiếm_hữu trọn_vẹn con_em trang_trại_vậy quotkhông đùaquot xui lắm_điều rồ_dại đính_hôn quotlaurie chín_chắn lứa giờ_phút


In [129]:
# Apply for all .txt files in folder
# Or apply only for 1000 files first (set num_file or set None)

import os
import re
import pandas as pd
from tqdm import tqdm
from num2words import num2words as n2w
import spacy
from pyvi import ViTokenizer
import time

nlp = spacy.load('en_core_web_sm')

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

def load_vietnamese_proper_nouns(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(f.read().splitlines())

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[,.]', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'(\d+)', '<number>', text)
    return text

def remove_english_proper_nouns(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if not (token.ent_type_ == "PERSON")]
    return ' '.join(filtered_tokens)

def remove_vietnamese_proper_nouns(text, proper_nouns):
    for noun in proper_nouns:
        text = text.replace(noun, '')
    return text

def normalize_vietnamese_text(text):
    text = ViTokenizer.tokenize(text)
    text = re.sub(r'\u200B', '', text)
    return text.replace(' ', '')

def lemmatize_numbers(text):
    return re.sub(r'(\d+)', lambda x: n2w(int(x.group())), text)

def tokenize_vietnamese_text(text):
    return ViTokenizer.tokenize(text)

def remove_duplicates(tokens):
    seen = set()
    unique_tokens = []
    for token in tokens.split():
        if token not in seen:
            unique_tokens.append(token)
            seen.add(token)
    return ' '.join(unique_tokens)

def preprocess_text(text, stop_words, proper_nouns):
    text = normalize_text(text)
    text = remove_english_proper_nouns(text)
    text = remove_vietnamese_proper_nouns(text, proper_nouns)
    tokens = tokenize_vietnamese_text(text)
    tokens = [word for word in tokens.split() if word not in stop_words]
    tokens = remove_duplicates(' '.join(tokens))
    text = lemmatize_numbers(tokens)
    return text

def process_text_file(input_file, output_file, stop_words, proper_nouns):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Chia nhỏ nội dung nếu nó dài hơn 1 triệu ký tự
        max_length = 1000000
        if len(content) > max_length:
            parts = [content[i:i + max_length] for i in range(0, len(content), max_length)]
            processed_text = ''
            for part in parts:
                processed_text += preprocess_text(part, stop_words, proper_nouns) + '\n'
        else:
            processed_text = preprocess_text(content, stop_words, proper_nouns)

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(processed_text)
            
    except Exception as e:
        print(f"Error processing {input_file}: {e}")
        raise  # Dừng xử lý và nâng lỗi

input_folder_path = '/kaggle/working/cleaned_header_footer_books'
output_folder_path = '/kaggle/working/preprocessed_books'
stopwords_path = '/kaggle/input/vietnamesestopwordstxt/vietnamese-stopwords.txt'
vietnamese_proper_nouns_path = '/kaggle/input/d/quynhnhitran02/vietnamese-proper-nouns/Vietnamese_Proper_Nouns.txt'

start_time = time.time()

num_files = 500  # Thay đổi giá trị này thành None để xử lý toàn bộ file  # 1 file thì xử lý tầm 9s, 500 file tầm 1,25h => ?_?
input_files = [f for f in os.listdir(input_folder_path) if f.endswith('.txt')]

if num_files is not None and num_files < len(input_files):
    input_files = input_files[:num_files]

stop_words = set()
vietnamese_proper_nouns = set()

with tqdm(total=2, desc="Loading resources") as pbar:
    stop_words = load_stopwords(stopwords_path)
    pbar.update(1)
    
    vietnamese_proper_nouns = load_vietnamese_proper_nouns(vietnamese_proper_nouns_path)
    pbar.update(1)

with tqdm(total=len(input_files), desc="Processing files") as pbar:
    for input_file in input_files:
        input_file_path = os.path.join(input_folder_path, input_file)
        output_file_path = os.path.join(output_folder_path, input_file)
        
        try:
            process_text_file(input_file_path, output_file_path, stop_words, vietnamese_proper_nouns)
        except Exception as e:
            print(f"Stopping processing due to error: {e}")
            break  # Dừng xử lý khi có lỗi, để check các trưởng hợp ngoại lệ, có thể pass các case ngoại lệ vì data nhiều được không?

        pbar.update(1)

end_time = time.time()
print(f"Processing completed in {end_time - start_time:.2f} seconds.")


Processing files: 100%|██████████| 500/500 [47:54<00:00,  5.75s/it]  

Processing completed in 2874.58 seconds.


In [139]:
processed_4cogai_path = '/kaggle/working/preprocessed_books/Những phụ nữ mở nước đâu tiên - Sử Gia Trần Gia Phụng.txt'

processed_4cogai = read_output_file(processed_4cogai_path)

print(processed_4cogai)

sử_gia trần gia phụng phụ_nữ việt_nam đầu_tiên hai trưng tiểu_sử sự_nghiệp ý_kiến hầu_như nêu lịch_sử thế_giới khởi_nghĩa chống ngoại_xâm < number > công_nguyên bậc nữ lưu đứng giành độc_lập đất_nước anh_hùng jeanne d arc pháp thế_kỷ công_chúa huyền trân ngọc vạn khoa liên_kết_đẩy lui xâm_lăng quân mông cổ giao_hảo đại việt tốt_đẹp tân sửu sứ_giả phẩm vật thăm_viếng ngoại_giao thái thượng hoàng nhân tông đi xuất_gia đi_tu thăm lễ du ngoạn âm_lịch vua chế mân jaya simhavarman iv trị_vì nguyên thái_tử bổ đích harijit đầu indravarman xi thời kháng_nguyên iii già nắm trọng_trách điều_khiển chỉ_huy chiêm_đẩy lực_lượng toa đô sogatu gặp_gỡ hứa gả con_gái có_thể bang_giao chiêm bền_vững hôn_nhân nầy phản_bác triều đình ta quan_niệm khắc biệt chủng_tộc quan ngăn_trở mãi quyết châu ô bắc sính cưới diễn bính ngọ đinh mùi thành thuận lẽ lý thành_hóa châu_hóa thay_đổi dạy_dỗ ngày_nay nam tỉnh quảng_trị thừa thiên hóa quảng_nam tổng_cộng vùng_đất km phong hậu paramecvari đám_cưới từ_trần giàn_hỏa_t

archieve data after preprocess

In [130]:
# Tạo folder để lưu các folder được nén => save nhanh hơn

output_folder = '/kaggle/working/save'
os.makedirs(output_folder, exist_ok=True)

In [131]:
input_folder = '/kaggle/working/cleaned_header_footer_books'

output_zip = os.path.join(output_folder, 'cleaned_header_footer_books.zip')

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', input_folder)

print(f"Folder {input_folder} has been zipped to {output_zip}")

Folder /kaggle/working/cleaned_header_footer_books has been zipped to /kaggle/working/save/cleaned_header_footer_books.zip


In [132]:
# Zip folder preprocessed_books

input_folder = '/kaggle/working/preprocessed_books'

output_zip = os.path.join(output_folder, 'preprocessed_books.zip')

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', input_folder)

print(f"Folder {input_folder} has been zipped to {output_zip}")


Folder /kaggle/working/preprocessed_books has been zipped to /kaggle/working/save/preprocessed_books.zip


Chuẩn bị input cho model

In [ ]:
# Step 1: Chuẩn bị dl

# Chọn ra 500 file .txt có dung lượng lớn nhất => Đảm bảo 500 file này có nội dung đủ nhiều và phong phú cho model học (Test trước với 500 files)

input_dir = '/kaggle/working/preprocessed_books'
files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.txt')]
largest_files = sorted(files, key=os.path.getsize, reverse=True)[:500]


In [ ]:
# Load file
texts = []
for file in tqdm(largest_files, desc="Loading text files"):
    with open(file, 'r', encoding='utf-8') as f:
        texts.append(f.read())

In [ ]:
texts[:1]

Train model
(Phần này em tham khảo từ https://github.com/Narius2030/Vietnamese-Text-Generator/blob/main/text-generator-train.ipynb)
- Em bị crash ở phần train model, model.fit. Mặc dù đã thử đổi batch size = 32, 64, 128 hay 256, tăng và giảm epoch, giảm số files từ 50->100. Nhưng vẫn bị crash :<
- Accelerator ở Kaggle em chọn GPU T4 x2.

In [ ]:
# filter several punctuations in texts
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ')
tokenizer.fit_on_texts(texts)

In [ ]:
# Print the length of the vocabulary
print(len(tokenizer.word_index))
tokenizer.word_index

In [ ]:
# tạo batch sequences từ dữ liệu
def create_input_generator(text_data, batch_size=10000):
    for i in range(0, len(text_data), batch_size):
        batch_texts = text_data[i:i + batch_size]
        input_sequences = []
        for line in batch_texts:
            token_list = tokenizer.texts_to_sequences([line])[0]
            for j in range(1, len(token_list)):
                n_gram_sequence = token_list[:j+1]
                input_sequences.append(n_gram_sequence)
        yield input_sequences

In [ ]:
# Generator sẽ tạo batch các sequences để giảm tải bộ nhớ
def create_sequences_in_batches(text_data, batch_size=10000):
    sequences_digit = []
    for batch in create_input_generator(text_data, batch_size):
        sequences_digit.extend(batch)
    return sequences_digit

In [ ]:
# Sử dụng batch size để tránh crash
batch_size = 10000
sequences_digit = create_sequences_in_batches(texts, batch_size)
print(sequences_digit[0:5])
print(len(sequences_digit))


In [ ]:
# Padding các sequences để có cùng chiều dài
max_sequence_len = 500
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_sequence_len, padding='pre'))


In [15]:
# Xác định kích thước từ vựng
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)


Vocabulary size: 77812


In [16]:
# Tách dữ liệu thành X (input) và y (output)
X = input_sequences[:,:-1]
y = input_sequences[:,-1]


In [17]:
# Chuyển y thành dạng one-hot
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)


In [18]:
# Kiểm tra dữ liệu X và y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (379437, 499)
Shape of y: (379437, 77812)


In [19]:
print(X)

[[    0     0     0 ...     0     0  1048]
 [    0     0     0 ...     0  1048   747]
 [    0     0     0 ...  1048   747  1124]
 ...
 [ 1147  3241 25807 ...  5433  6848  1422]
 [ 3241 25807  1510 ...  6848  1422 77811]
 [25807  1510   814 ...  1422 77811 12598]]


In [20]:
print('The length of a one-hot vector y: ', len(y[0]))
print('The collection of y vectors for all sentences', y)

The length of a one-hot vector y:  77812
The collection of y vectors for all sentences [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [21]:
# Xây dựng mô hình LSTM
model = tf.keras.models.Sequential()

In [22]:
# Xây dựng mô hình LSTM
model = tf.keras.models.Sequential()

# Embedding layer
model.add(tf.keras.layers.Embedding(vocab_size, 50, input_length=max_sequence_len-1))

# Batch normalization và LSTM layers
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(512, return_sequences=True))
model.add(tf.keras.layers.LSTM(512))

# Dense layers
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
# Compile mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train model
summary = model.fit(X, y, batch_size=128, epochs=10)


In [ ]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)[0]
        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word
    return seed_text

In [ ]:
seed_text = "Thời gian"
next_words = 10
generated_text = generate_text(seed_text, next_words, max_sequence_len)
print(generated_text)